# Context Manager Control Flow

In [1]:
import sys
sys.version

'3.6.1 |Continuum Analytics, Inc.| (default, Mar 22 2017, 19:25:17) \n[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]'

## Creating a context manager protocol

### Let's figure out control flow

In [2]:
class Foo():
    def __init__(self):
        print('__init__ called')
        self.init_var = 0
        
    def __enter__(self):
        print('__enter__ called')
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('__exit__ called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_traceback}')
            
    def add_two(self):
        self.init_var += 2

In [3]:
my_object = Foo()

__init__ called


In [4]:
my_object.init_var

0

In [5]:
my_object.add_two()
my_object.init_var

2

In [6]:
# regular flow without exceptions
with my_object as obj:
    print('inside with statement body')

__enter__ called
inside with statement body
__exit__ called


In [7]:
# what can we access in the object that is returned inside with statement context
with my_object as obj:
    print(obj.init_var)

__enter__ called
2
__exit__ called


In [8]:
# adding 2 to the var
my_object.add_two()
with my_object as obj:
    print(obj.init_var)

__enter__ called
4
__exit__ called


In [9]:
# using a new instance in context expression
with Foo() as obj:
    print(obj.init_var)

__init__ called
__enter__ called
0
__exit__ called


In [10]:
# raising exceptions within block
with my_object as obj:
    print('inside with statement body')
    raise Exception('exception raised').with_traceback(None)

__enter__ called
inside with statement body
__exit__ called
exc_type: <class 'Exception'>
exc_value: exception raised
exc_traceback: <traceback object at 0x102d45a48>


Exception: exception raised

In [11]:
# try to handle exception
try:
    with my_object as obj:
        print('inside with statement body')
        raise Exception('exception raised').with_traceback(None)
except Exception as e:
    print('handling exception')
    print(e)
finally:
    print('Finally section')

__enter__ called
inside with statement body
__exit__ called
exc_type: <class 'Exception'>
exc_value: exception raised
exc_traceback: <traceback object at 0x102d3e248>
handling exception
exception raised
Finally section


### There is probably a better way to do this, come back to it

In [12]:
# with statement within a with statement... with-ception
with my_object as obj:
    print('inside first context')
    with my_object as obj2:
        raise Exception('exception raised inner most block').with_traceback(None)
    
    print('a')

__enter__ called
inside first context
__enter__ called
__exit__ called
exc_type: <class 'Exception'>
exc_value: exception raised inner most block
exc_traceback: <traceback object at 0x102d21bc8>
__exit__ called
exc_type: <class 'Exception'>
exc_value: exception raised inner most block
exc_traceback: <traceback object at 0x102d21bc8>


Exception: exception raised inner most block

In [13]:
my_object.init_var

4

In [14]:
# how does variable context change?
# with statement within a with statement... with-ception
with my_object as obj:
    my_object.add_two()
    print('inside first context')
    with my_object as obj2:
        raise Exception('exception raised inner most block').with_traceback(None)
    
    print('a')

__enter__ called
inside first context
__enter__ called
__exit__ called
exc_type: <class 'Exception'>
exc_value: exception raised inner most block
exc_traceback: <traceback object at 0x102dc33c8>
__exit__ called
exc_type: <class 'Exception'>
exc_value: exception raised inner most block
exc_traceback: <traceback object at 0x102dc33c8>


Exception: exception raised inner most block

In [15]:
my_object.init_var

6

### Since we created my_object outside of a with statement, it can be changed. Instance exists

### If we create a new context expression variable in our with statement, it only exists inside the with's scope

---

## How can we handle Exceptions?

#### If the __exit__() method returns True, the exception can be considered handled... so let's test that out

In [16]:
class Foo2():
    def __init__(self):
        print('__init__ called')
        self.init_var = 0
        
    def __enter__(self):
        print('__enter__ called')
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('__exit__ called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_traceback}')
            print('exception handled')
        # return True to handle exception...
        return True
            
    def add_two(self):
        self.init_var += 2

In [17]:
# using a new instance in context expression
with Foo2() as obj:
    print(obj.init_var)
    raise Exception('exception raised').with_traceback(None)

__init__ called
__enter__ called
0
__exit__ called
exc_type: <class 'Exception'>
exc_value: exception raised
exc_traceback: <traceback object at 0x102b61248>
exception handled
